Topic
0. open
1. review
  
   1.1 1, 2일차 내용에 대한 review( naver 영화 한줄평을 이용 )
2. topic

   2.1 정수 인코딩( Integer encoding )

   2.2 원-핫 인코딩( One-hot encoding )
   
   2.3 Byte Pair Encoding, BPE )
   
   2.4 데이터 분리( Splitting Data )
   
3. Q & A
4. close

# 1. Review

# Naver 영화 한줄평 crawling & Test preprocessing


In [1]:
import requests
from bs4 import BeautifulSoup

# 한줄평 crawling

In [2]:
url = 'https://movie.naver.com/movie/bi/mi/basic.nhn?code=187321#'

In [3]:
resp = requests.get( url )
html = BeautifulSoup( resp.content, 'lxml' )

In [4]:
score_result = html.find( 'div', { 'class' : 'score_result' } )
lis = score_result.findAll( 'li' )

In [6]:
comments = []
for li in lis:
    review_text = li.find( 'p' ).getText()
    comments.append( review_text.strip() )
print( comments )

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


In [7]:
for comment in comments:
    print( comment + '\n' )

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.

충무로: 이거 어케하는거냐?

아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.

촬영감독의 영혼까지 갈아넣은 마스터피스

오스카 작품상 탔어도 할말 없었을것 같다.



### crawling 결과 파일 저장 / 읽기

In [8]:
with open( 'comment_1917.txt', 'w', encoding = 'utf-8' ) as f:
    for comment in comments:
        f.write( comment + '\n' )

In [9]:
movie_comments = []
with open( 'comment_1917.txt', 'r', encoding = 'utf-8' ) as f:
    lines = f.readlines()
    for line in lines:
        movie_comments.append( line.rstrip( '\n' ) )
print( movie_comments )

['이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.', '충무로: 이거 어케하는거냐?', '아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.', '촬영감독의 영혼까지 갈아넣은 마스터피스', '오스카 작품상 탔어도 할말 없었을것 같다.']


### 파이썬 리스트 내용을 하나의 문자열로 변경

In [10]:
movie_comment = ''
for comment in movie_comments:
    movie_comment += comment 
print( movie_comment )

이 영화는 미쳤다. 넷플릭스가 일상화된 시대에 극장이 존재해야하는 이유를 증명해준다.충무로: 이거 어케하는거냐?아카데미에서 촬영상, 음향효과상, 시각효과상을 받은 이유가 고스란히 녹아있는 영화. IMAX로 관람하는걸 추천한다.촬영감독의 영혼까지 갈아넣은 마스터피스오스카 작품상 탔어도 할말 없었을것 같다.


# Text preprocessing